<a href="https://colab.research.google.com/github/ipeirotis/autoencoders_census/blob/main/5_outlier_most_informative_variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
!rm -rf autoencoders_census
!git clone https://github.com/ipeirotis/autoencoders_census.git
%cd autoencoders_census

Cloning into 'autoencoders_census'...
remote: Enumerating objects: 452, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 452 (delta 48), reused 2 (delta 2), pack-reused 366
Receiving objects: 100% (452/452), 47.94 MiB | 17.42 MiB/s, done.
Resolving deltas: 100% (262/262), done.
/content/autoencoders_census/autoencoders_census


In [2]:
from google.colab import auth

# Login using the account that has access to the Google project
# in order to access the resources for the project
auth.authenticate_user()

In [3]:
import gcsfs

gcs_dataset_path = "gs://autoencoder_census_models/data_with_reconstructed_errors.csv"

# Define the local path where the model will be downloaded
local_dataset_path = "5%_outlier_pattern_classification.csv"

# Create a GCS filesystem object using your project ID
fs = gcsfs.GCSFileSystem(project='autoencoder-census')


with fs.open(gcs_dataset_path, 'rb') as gcs_file:
    with open(local_dataset_path, 'wb') as local_file:
        local_file.write(gcs_file.read())

In [4]:
import pandas as pd

outlier_analysis_df = pd.read_csv(local_dataset_path)
outlier_analysis_df.head()

,Unnamed: 0,age,sex,grade,Hispanic_or_Latino,race,obese,overweight,sexual_identity,sex/sexual_contacts,...,bmipct_cat.1,custom_error,custom_rank,mae_error,mae_rank,mse_error,mse_rank,cce_error,cce_rank,error_rank
0,100,16 years old,NaN,11th,Black or African American,Black or African American,NaN,NaN,Heterosexual (straight),Males,...,missing,0.806967,1.000000,0.216840,1.000000,0.115637,1.000000,593.03210,1.000000,1.000000
1,7757,14 years old,Female,9th,All other races,Asian,NaN,NaN,Gay or lesbian,Females,...,missing,0.768707,0.999865,0.213092,0.999932,0.109196,0.999932,588.97630,0.999865,0.999898
2,6848,18 years old or older,Male,11th,Black or African American,Black or African American,obese,not overweight,Not sure,NaN,...,normal,0.781711,0.999932,0.205603,0.999729,0.108340,0.999865,587.65173,0.999797,0.999831
3,7242,12 years old or younger,Female,12th,Hispanic/Latino,Hispanic/Latino,NaN,NaN,Bisexual,Males,...,missing,0.762938,0.999729,0.205574,0.999661,0.105912,0.999661,589.49900,0.999932,0.999746
4,3969,16 years old,Male,9th,All other races,Am Indian / Alaska Native,NaN,NaN,Bisexual,NaN,...,missing,0.762254,0.999661,0.206458,0.999797,0.105946,0.999729,586.44090,0.999594,0.999695


In [5]:
filtered_columns = [col for col in outlier_analysis_df.columns if col.endswith('.1')]
outlier_analysis_df = outlier_analysis_df.drop(columns=filtered_columns)

In [6]:
outlier_analysis_df.shape

(14765, 108)

In [7]:
data_index = outlier_analysis_df['Unnamed: 0']
columns_to_drop = ['Unnamed: 0', 'custom_error', 'custom_rank', 'mae_error', 'mae_rank', 'mse_error', 'mse_rank', 'cce_error', 'cce_rank']
outlier_analysis_df = outlier_analysis_df.drop(columns=columns_to_drop)

In [8]:
#top_quantile = outlier_analysis_df['error_rank'].quantile(0.9)
#bottom_quantile = outlier_analysis_df['error_rank'].quantile(0.1)

#outlier_analysis_df['error_rank'] = 0
#outlier_analysis_df.loc[outlier_analysis_df['error_rank'] >= top_quantile, 'error_rank'] = 1
#outlier_analysis_df.loc[outlier_analysis_df['error_rank'] <= bottom_quantile, 'error_rank'] = 2


In [9]:
#from sklearn.preprocessing import LabelEncoder

#label_encoder = LabelEncoder()
#categorical_columns = outlier_analysis_df.drop('error_rank', axis=1).columns
#for col in categorical_columns:
    #outlier_analysis_df[col] = label_encoder.fit_transform(outlier_analysis_df[col])

In [10]:
outlier_analysis_df = pd.get_dummies(outlier_analysis_df, columns=outlier_analysis_df.drop('error_rank', axis=1).columns, drop_first=True)

In [11]:
correlation = outlier_analysis_df.corr()
display(correlation)

,error_rank,age_13 years old,age_14 years old,age_15 years old,age_16 years old,age_17 years old,age_18 years old or older,sex_Male,grade_11th,grade_12th,...,stheight_cat_normal,stheight_cat_top-extreme,stweight_cat_missing,stweight_cat_normal,stweight_cat_top-extreme,bmi_cat_missing,bmi_cat_normal,bmi_cat_top-extreme,bmipct_cat_missing,bmipct_cat_normal
error_rank,1.000000,0.000092,-0.154722,-0.085851,0.005925,0.077206,0.144272,0.033376,0.039229,0.096190,...,-0.093149,0.035758,0.082328,-0.112659,0.070268,0.071509,-0.098460,0.060484,0.082328,-0.051323
age_13 years old,0.000092,1.000000,-0.014944,-0.021879,-0.022290,-0.021979,-0.014375,0.001416,-0.017855,-0.016882,...,-0.014766,-0.007992,0.025781,-0.015799,-0.009385,0.026724,-0.017372,-0.008765,0.025781,-0.015857
age_14 years old,-0.154722,-0.014944,1.000000,-0.219102,-0.223217,-0.220111,-0.143961,-0.028918,-0.218811,-0.210425,...,0.018272,-0.051625,0.001450,0.023289,-0.044860,0.000040,0.023054,-0.043011,0.001450,0.010435
age_15 years old,-0.085851,-0.021879,-0.219102,1.000000,-0.326805,-0.322257,-0.210767,-0.009263,-0.313634,-0.308401,...,0.024191,-0.039153,-0.002128,0.031954,-0.048799,-0.002897,0.022634,-0.032639,-0.002128,0.008422
age_16 years old,0.005925,-0.022290,-0.223217,-0.326805,1.000000,-0.328309,-0.214726,0.011449,0.295558,-0.301547,...,0.007308,0.020215,-0.021729,0.017329,0.003022,-0.018026,0.017169,-0.001878,-0.021729,0.021766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bmi_cat_missing,0.071509,0.026724,0.000040,-0.002897,-0.018026,-0.033470,0.003617,-0.011000,-0.030958,-0.035766,...,-0.750039,-0.071082,0.979063,-0.770132,-0.083479,1.000000,-0.801315,-0.077964,0.979063,-0.771264
bmi_cat_normal,-0.098460,-0.017372,0.023054,0.022634,0.017169,0.005128,-0.027507,0.004301,0.014103,-0.000276,...,0.608084,0.046291,-0.788527,0.859567,-0.318035,-0.801315,1.000000,-0.529201,-0.788527,0.602308
bmi_cat_top-extreme,0.060484,-0.008765,-0.043011,-0.032639,-0.001878,0.040290,0.041340,0.009890,0.020915,0.051038,...,0.049240,0.024245,-0.072600,-0.341597,0.653496,-0.077964,-0.529201,1.000000,-0.072600,0.095175
bmipct_cat_missing,0.082328,0.025781,0.001450,-0.002128,-0.021729,-0.035780,0.001370,-0.020753,-0.032788,-0.036594,...,-0.766078,-0.072602,1.000000,-0.786601,-0.085264,0.979063,-0.788527,-0.072600,1.000000,-0.787757


In [12]:
sorted_correlation = correlation['error_rank'].sort_values(ascending=False)

most_correlated_columns = sorted_correlation.index[1:11]
print("Most correlated columns with 'error_rank':")
print(most_correlated_columns)

Most correlated columns with 'error_rank':
Index(['ever_cigarette_use_Yes', 'ever_sexual_intercourse_Yes',
       'sexpart2_Opposite sex only', 'sexpart_Opposite sex only',
       'electronic_vapor_product_use_Yes', 'alcohol/drugs_at_sex_No',
       'condom_use_No', 'difficulty_concentrating_Yes',
       'ever_marijuana_use_100 or more times', 'alcohol/drugs_at_sex_Yes'],
      dtype='object')


In [45]:
X = outlier_analysis_df.drop('error_rank', axis = 1)
y = outlier_analysis_df['error_rank']

In [46]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(outlier_analysis_df, test_size=0.20,random_state=42)
df_train = df_train[list(outlier_analysis_df.columns)]
df_test = df_test[list(outlier_analysis_df.columns)]

In [47]:
X_train, y_train = df_train.drop('error_rank',axis=1), df_train['error_rank']
X_test, y_test = df_test.drop('error_rank',axis=1), df_test['error_rank']

In [48]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((11812, 418), (11812,), (2953, 418), (2953,))

In [49]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor()
regressor.fit(outlier_analysis_df.drop('error_rank', axis=1), outlier_analysis_df['error_rank'])
feature_importances = regressor.feature_importances_

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_regressor = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

In [ ]:
feature_importances = best_estimator.feature_importances_

In [ ]:
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

In [19]:
RFimportances_df = pd.DataFrame({'Feature': outlier_analysis_df.drop('error_rank', axis=1).columns, 'Importance': feature_importances})
RFimportances_df = RFimportances_df.sort_values(by='Importance', ascending=False)
RFimportances_df.head(10)

,Feature,Importance
181,initiation_of_marijuana_use_Never tried marijuana,0.277155
112,ever_cigarette_use_Yes,0.082632
122,smoking_amounts_per_day_Did not smoke cigarettes,0.071965
149,initiation_of_alcohol_use_Never drank alcohol,0.057113
169,largest_number_of_drinks_Did not drink in past...,0.025156
252,birth_control_pill_use_Never had sex,0.017053
158,source_of_alcohol_Did not drink in past 30 days,0.014238
103,sad_or_hopeless_Yes,0.014075
225,ever_sexual_intercourse_Yes,0.013761
33,sexpart_Opposite sex only,0.012293


In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r_squared = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r_squared)

Mean Squared Error: 0.01539667989986153
Root Mean Squared Error: 0.12408335867416521
R-squared: 0.8112151423322217


In [21]:
coefficients = model.coef_
feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Coefficient': coefficients})
feature_importance = feature_importance.reindex(feature_importance['Coefficient'].abs().sort_values(ascending=False).index)

sorted_feature_importance = feature_importance.sort_values(by='Coefficient', ascending=False)
pd.options.display.float_format = '{:.4f}'.format
top_10_largest = sorted_feature_importance.head(10)
top_10_smallest = sorted_feature_importance.tail(10)

print("Top 10 Largest Coefficients:")
print(top_10_largest)
print("\nTop 10 Smallest Coefficients:")
print(top_10_smallest)

Top 10 Largest Coefficients:
                                     Feature      Coefficient
14            race_Black or African American 77028246308.9193
11        Hispanic_or_Latino_Hispanic/Latino 61728874360.0513
416                       bmipct_cat_missing 52081866806.0393
22   sexual_identity_Heterosexual (straight) 42198818414.1848
12                  Hispanic_or_Latino_White 22530646932.3960
27                      sexid_Gay or Lesbian 16100106985.4447
33                 sexpart_Opposite sex only 14210490937.5963
29                            sexid_Not Sure 12966090139.7856
23                  sexual_identity_Not sure  1662225416.5584
36      sexpart2_Same sex only or both sexes           0.1853

Top 10 Smallest Coefficients:
                                          Feature       Coefficient
413                               bmi_cat_missing           -0.2391
410                          stweight_cat_missing -11144327384.4634
35                     sexpart2_Opposite sex only -142

In [34]:
top_5_percent_outliers = pd.read_csv(local_dataset_path)
top_5_percent = int(0.05 * len(top_5_percent_outliers))
top_5_percent_df = top_5_percent_outliers.head(top_5_percent)
top_5_percent_df.shape

(738, 206)

In [35]:
filtered_columns = [col for col in top_5_percent_df.columns if col.endswith('.1')]
top_5_percent_df = top_5_percent_df.drop(columns=filtered_columns)

In [36]:
top_5_percent_df = pd.get_dummies(top_5_percent_df, columns=top_5_percent_df.drop('error_rank', axis=1).columns, drop_first=True)

In [37]:
correlation = top_5_percent_df.corr()
sorted_correlation = correlation['error_rank'].sort_values(ascending=False)
most_correlated_columns = sorted_correlation.index[1:11]
print("Most correlated columns with 'error_rank':")
print(most_correlated_columns)

Most correlated columns with 'error_rank':
Index(['illegal_injected_drug_use_2 or more times',
       'ever_synthetic_marijuana_use_20 to 39 times', 'seat_belt_use_Never',
       'initiation_of_marijuana_use_8 years old or younger',
       'ever_methamphetamine_use_3 to 9 times', 'made_a_suicide_plan_Yes',
       'ever_steroid_use_3 to 9 times',
       'oral_health_care_More than 24 months ago',
       'injurious_suicide_attempt_Yes', 'food_allergies_Not sure'],
      dtype='object')


In [38]:
regressor = RandomForestRegressor()
regressor.fit(top_5_percent_df.drop('error_rank', axis=1), top_5_percent_df['error_rank'])
feature_importances = regressor.feature_importances_

In [39]:
RFimportances_df = pd.DataFrame({'Feature': top_5_percent_df.drop('error_rank', axis=1).columns, 'Importance': feature_importances})
RFimportances_df = RFimportances_df.sort_values(by='Importance', ascending=False)
RFimportances_df.head(10)

,Feature,Importance
960,illegal_injected_drug_use_2 or more times,0.0404
859,smoking_amounts_per_day_Did not smoke cigarettes,0.0345
951,ever_synthetic_marijuana_use_20 to 39 times,0.0142
842,made_a_suicide_plan_Yes,0.0139
1007,fruit_eating_1 to 3 times,0.0130
1120,food_allergies_Not sure,0.0107
916,initiation_of_marijuana_use_8 years old or you...,0.0102
957,ever_steroid_use_3 to 9 times,0.0101
765,sexid_Heterosexual,0.0096
775,seat_belt_use_Never,0.0096


In [40]:
df_train, df_test = train_test_split(top_5_percent_df, test_size=0.20,random_state=42)
df_train = df_train[list(top_5_percent_df.columns)]
df_test = df_test[list(top_5_percent_df.columns)]
X_train, y_train = df_train.drop('error_rank',axis=1), df_train['error_rank']
X_test, y_test = df_test.drop('error_rank',axis=1), df_test['error_rank']

In [41]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r_squared = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r_squared)

Mean Squared Error: 0.00018588680539979798
Root Mean Squared Error: 0.013634031150023018
R-squared: 0.1776188654689692


In [42]:
coefficients = model.coef_
feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Coefficient': coefficients})
feature_importance = feature_importance.reindex(feature_importance['Coefficient'].abs().sort_values(ascending=False).index)

sorted_feature_importance = feature_importance.sort_values(by='Coefficient', ascending=False)
pd.options.display.float_format = '{:.4f}'.format
top_10_largest = sorted_feature_importance.head(10)
top_10_smallest = sorted_feature_importance.tail(10)

print("Top 10 Largest Coefficients:")
print(top_10_largest)
print("\nTop 10 Smallest Coefficients:")
print(top_10_smallest)

Top 10 Largest Coefficients:
                                            Feature  Coefficient
1120                        food_allergies_Not sure       0.0055
960       illegal_injected_drug_use_2 or more times       0.0050
780      riding_with_a_drinking_driver_4 or 5 times       0.0048
771                           sexpart_Same sex only       0.0047
8                                     Unnamed: 0_73       0.0045
957                   ever_steroid_use_3 to 9 times       0.0044
13                                    Unnamed: 0_83       0.0043
929                  ever_inhalant_use_1 or 2 times       0.0041
781   riding_with_a_drinking_driver_6 or more times       0.0039
4                                     Unnamed: 0_31       0.0039

Top 10 Smallest Coefficients:
                                               Feature  Coefficient
911                    ever_marijuana_use_3 to 9 times      -0.0034
2                                        Unnamed: 0_15      -0.0036
1095                 

In [43]:
!pip install scikeras

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import tensorflow as tf

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def create_model(learning_rate=0.001):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

keras_regressor = KerasRegressor(build_fn=create_model)

param_grid = {
    'epochs': [10, 30, 50],
    'batch_size': [32, 64]
}

grid = GridSearchCV(estimator=keras_regressor, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)

grid_result = grid.fit(X_train, y_train)

best_params = grid_result.best_params_
best_estimator = grid_result.best_estimator_

print("Best Parameters:", best_params)
print("Best MSE:", -grid_result.best_score_)



In [226]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.014022 using {'batch_size': 64, 'epochs': 10}


In [ ]:
import keras
final_model = Sequential()
final_model.add(Dense(31, input_dim=418, kernel_initializer='uniform',
          activation='relu'))
final_model.add(Dense(106, activation='relu'))
final_model.add(Dense(1, kernel_initializer='uniform',
          activation='sigmoid'))

opt = keras.optimizers.Adam(learning_rate=0.1)
final_model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])

final_model.fit(X_train, y_train, epochs=10, batch_size=64)

In [231]:
! pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.9/547.9 kB 4.1 MB/s eta 0:00:00


In [235]:
import shap

In [238]:
explainer = shap.Explainer(final_model, X_train, max_evals=1000)
shap_values = explainer(X_test)
shap.summary_plot(shap_values, X_test)

AttributeError: ignored

In [193]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

X = outlier_analysis_df.drop('error_rank', axis = 1)
y = outlier_analysis_df['error_rank']
shuffled_X, shuffled_y = shuffle(X, y, random_state=42)
xTrain, xTest, yTrain, yTest = train_test_split(shuffled_X,shuffled_y, test_size = 0.2, random_state = 0)

In [194]:
from keras.models import Model
from keras.layers import Dense, Input, Dropout

def NN_model(input):

    input_ = Input(shape=input.shape[1:])
    x = Dense(1600, activation='relu')(input_)
    x = Dropout(0.5)(x)
    x = Dense(1600, activation='relu')(input_)
    x = Dropout(0.5)(x)
    x = Dense(1600, activation='relu')(input_)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='softmax')(x)

    model = Model(input_, output)
    return model

In [195]:
model = NN_model(xTrain)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 418)]             0         
                                                                 
 dense_10 (Dense)            (None, 1600)              670400    
                                                                 
 dropout_8 (Dropout)         (None, 1600)              0         
                                                                 
 dense_11 (Dense)            (None, 1)                 1601      
                                                                 
Total params: 672001 (2.56 MB)
Trainable params: 672001 (2.56 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [196]:
import keras
model.compile(keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [197]:
from keras.callbacks import EarlyStopping

batch_size = 32
es = EarlyStopping(patience=5, verbose=1, min_delta=0.001, monitor='val_loss', mode='auto')
model.fit(x=xTrain, y=yTrain, batch_size=batch_size, epochs=20,validation_split=0.2 , callbacks = [es])

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


293/296 [============================>.] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0666e-04

/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


296/296 [==============================] - 5s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0583e-04 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
296/296 [==============================] - 4s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0583e-04 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/20
296/296 [==============================] - 4s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0583e-04 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/20
296/296 [==============================] - 4s 14ms/step - loss: 0.0000e+00 - accuracy: 1.0583e-04 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/20
296/296 [==============================] - 4s 14ms/step - loss: 0.0000e+00 - accuracy: 1.0583e-04 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/20
296/296 [==============================] - 4s 14ms/step - loss: 0.0000e+00 - accuracy: 1.0583e-04 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6: early stopping
